In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('strokeData.csv')
print("type(df)-->",type(df))

df.head()

type(df)--> <class 'pandas.core.frame.DataFrame'>


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [2]:
df = df.drop_duplicates()

In [3]:
mean1 = df['bmi'].mean()
print(mean1)
#df = df.fillna({'bmi': mean1})
df = df.interpolate()

28.893236911794666


/tmp/ipykernel_26921/3289592859.py:4: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate()


In [4]:
df.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [5]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
df['ever_married']= label_encoder.fit_transform(df['ever_married'])
df['work_type']= label_encoder.fit_transform(df['work_type'])
df['Residence_type']=label_encoder.fit_transform(df['Residence_type'])
df['smoking_status']=label_encoder.fit_transform(df['smoking_status'])
df['gender']=label_encoder.fit_transform(df['gender'])

df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,2,1,228.69,36.60,1,1
1,51676,0,61.0,0,0,1,3,0,202.21,34.55,2,1
2,31112,1,80.0,0,1,1,2,0,105.92,32.50,2,1
3,60182,0,49.0,0,0,1,2,1,171.23,34.40,3,1
4,1665,0,79.0,1,0,1,3,0,174.12,24.00,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,0,80.0,1,0,1,2,1,83.75,29.30,2,0
5106,44873,0,81.0,0,0,1,3,1,125.20,40.00,2,0
5107,19723,0,35.0,0,0,1,3,0,82.99,30.60,2,0
5108,37544,1,51.0,0,0,1,2,0,166.29,25.60,1,0


In [6]:
X = df["stroke"]
y = df.drop(['stroke'], axis=1)

In [7]:
from sklearn.model_selection import train_test_split,GridSearchCV

x_train, x_test, y_train, y_test = train_test_split(y, X, test_size=0.30, shuffle=True, random_state=0)

In [8]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)

In [9]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1457
           1       0.00      0.00      0.00        76

    accuracy                           0.95      1533
   macro avg       0.48      0.50      0.49      1533
weighted avg       0.90      0.95      0.93      1533



/workspaces/DataScienceProject/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/DataScienceProject/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/DataScienceProject/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [10]:
print(clf.score(x_train,y_train))
print(clf.score(x_test,y_test))

0.9516354487000279
0.9504240052185258


In [11]:
clf.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(max_depth=2, random_state=0)>

In [12]:
params={
    "n_estimators":[90,100,115,130],
    "criterion":['gini','entropy'],
    "max_depth":range(2,20,1),
    "min_samples_leaf":range(1,10,1),
    "min_samples_split":range(2,10,1),
    "max_features":['auto','log2']
}

In [13]:
grid_svc = GridSearchCV(estimator=clf, param_grid=params,cv=2,n_jobs=-1,verbose=3)

In [14]:
#grid_svc.fit(x_train, y_train)
#grid_svc.best_params_

In [15]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [16]:
classifiers=[['Naive Bayes :', GaussianNB()],['LogisticRegression :', LogisticRegression(max_iter = 1000)], ['DecisionTree :',DecisionTreeClassifier()]]

In [17]:
for name,classifier in classifiers:
    classifier = classifier
    classifier.fit(x_train, y_train.ravel())
    predictions = classifier.predict(x_test)
    print(name, accuracy_score(y_test, predictions))

Naive Bayes : 0.9178082191780822
LogisticRegression : 0.949119373776908
DecisionTree : 0.9093281148075668


In [24]:
from sklearn.ensemble import VotingClassifier
clf1 = GaussianNB()
clf2 = DecisionTreeClassifier()
clf3 = LogisticRegression()


    
vot_hard = VotingClassifier(estimators= classifiers, voting='hard')
vot_hard.fit(x_train, y_train)

VotingClassifier(estimators=[['Naive Bayes :', GaussianNB()],
                             ['LogisticRegression :',
                              LogisticRegression(max_iter=1000)],
                             ['DecisionTree :', DecisionTreeClassifier()]])

In [19]:
vot_soft = VotingClassifier(estimators = classifiers, voting ='soft')
vot_soft.fit(x_train, y_train)

VotingClassifier(estimators=[['Naive Bayes :', GaussianNB()],
                             ['LogisticRegression :',
                              LogisticRegression(max_iter=1000)],
                             ['DecisionTree :', DecisionTreeClassifier()]],
                 voting='soft')

In [20]:
print("Training data accuracy:", vot_hard.score(x_train, y_train))
print("Testing data accuracy", vot_hard.score(x_test, y_test))

print("Training data accuracy:", vot_soft.score(x_train, y_train))
print("Testing data accuracy", vot_soft.score(x_test, y_test))

Training data accuracy: 0.9608610567514677
Testing data accuracy 0.9452054794520548
Training data accuracy: 0.9644953871959743
Testing data accuracy 0.9445531637312459
